# Welcome to the Calvin at a Party Simulation!
![Whoah](jurassic_park_gif.gif "Whoah")

## Why?
As PhD Calvin, I felt I had to get into my role and crank out a simulation. After all, we know that all theoretical physicists really do is mess around in simulations ;)

## What does it do? 
This simulation simulates Calvin having fun at the bachelor party. There is a large bank of fun things that will randomly happen to him during the party with his bachelors. Each event will last for a certain number of time in the party and could involve Calvin taking 1 (or more!) drinks. The simulation will calculate Calvin's BAC over the course of the party and display it after each event occurs so we can track his "progress". The simulation ends once the length of the party in hours has been reached. Hope you enjoy!

![Cheers](cheers_gif.gif "Cheers")

## Simpy is used to run the simulation

In [1]:
import simpy
import random
import statistics

## Getting user input

In [2]:
def get_user_input():
    """ Get user inputs needed for the simulation """
    
    # Get Calvin's weight for the simulation (didn't know this lol)
    weight = float(input('Enter weight in pounds: '))
    weight *= 454.0 # convert to grams
    print(f'Weight is {weight} in grams')
    
    # How long we partying????
    party_hours = int(input('Enter party time in hours: '))
    party_minutes = party_hours * 60
    print(f'Party will be {party_minutes} minutes long')
    
    return (weight, party_minutes)

## Calculating Calvin's BAC

In [3]:
def calculate_bac(total_drinks, current_time):
    """ Use the Widmark formula to calculate BAC """
    
    if total_drinks == 0:
        return 0
    
    r = 0.68 # constant for males, assuming male bc bachelor party
    drink_alcohol = 14.0 # 14 grams of alcohol per 'standard' size drink
    total_alcohol = total_drinks * drink_alcohol
    bac_decay = 0
    bac_decay = current_time / 60 * 0.015
    bac = total_alcohol / (weight *  r) * 100 - bac_decay
    return bac

## Defining the Party
![mcconaughey](mcconaughey.gif "Alright Alright Alright")

In [4]:
class Party:
    """ Class representing the party environment. Includes the man himself and how many drinks he's consumed ;) """
    
    def __init__(self, env):
        self.env = env
        self.calvin = simpy.Resource(env, 1)
        self.drinks = 0
        self.things_done = 0
        
    def do_fun_thing(self, ft):
        """ Update current state of simulation based on the incoming Fun Thing """
        
        # Fill in placeholder #'s in description using random bachelor names
        temp_desc = ft.desc
        marker_count = ft.desc.count('#')
        for i in range(0, marker_count):
            temp_desc = temp_desc.replace('#', random.choice(bachelors), 1)
        print(temp_desc)
        
        # Update simulation time based on event duration
        yield self.env.timeout(ft.duration)
        
        # Update number of drinks consumed and calculate BAC
        self.drinks += ft.drinks
        bac = calculate_bac(self.drinks, self.env.now)
        print(f'    Current time is {self.env.now}')
        print(f'    You\'ve had {self.drinks} drinks')
        print(f'    Your current bac is {bac} ;)')
        self.things_done += 1

## Having some FUN
![fun](fun.gif "fun")

In [5]:
class FunThing:
    """ Class to represent any Fun Thing that could happen during the party """
    
    def __init__(self, duration, drinks, desc):
        self.duration = duration
        self.drinks = drinks
        self.desc = desc

In [6]:
def start_fun_thing(env, ft, party):
    """ Fire off a new Fun Thing to be processed by the Calvin resource """
    with party.calvin.request() as request:
        yield request
        yield env.process(party.do_fun_thing(ft))

In [11]:
bachelors = [ 'Logan', 'Vishal', 'Nathan', 'Naveen', 'Cal', 'Preston', 'Trevor', 'Veijay' ] # the all stars
fun_things = [
    FunThing(30, 0, '# challenges you to a round of Smash Bros'),
    FunThing(15, 0, 'You play Twister with # and #. You\'re all limbered up and ready to party'),
    FunThing(5, 1, '# pours you a shot of Kraken. Bottoms up!'),
    FunThing(30, 3, 'The whole party plays Boom. You get the boom cup ¯\_(ツ)_/¯'),
    FunThing(15, 0, 'You play some Rocket League with # and reach Grand Champion. Quit. Out. Of. Respect.'),
    FunThing(10, 0, '# starts telling you about his favorite craft beer. You drink one to appease him'),
    FunThing(20, 2, 'You go to grab a beer from the fridge and get Iced by #. Two for the price of one!'),
    FunThing(30, 1, 'You try a new beer with # and it\'s awesome'),
    FunThing(30, 1, 'The whole party plays Boom. # gets the boom cup'),
    FunThing(30, 2, 'Pokemon Sorry! Innocent right? Wrong. We play with Cal\'s drinking rules'),
    FunThing(60, 0, 'We play Settlers of Catan. Your monopoly on bricks brings home the dub'),
    
    
]

## Starting the party!

In [8]:
def run_party(env):
    """ Function to continually pick new Fun Things during the party """
    
    party = Party(env)

    while True:
        yield env.timeout(1)
        ft = random.choice(fun_things)
        env.process(start_fun_thing(env, ft, party))
        

In [9]:
weight, time_minutes = get_user_input()

Enter weight in pounds: 180
Weight is 81720.0 in grams
Enter party time in hours: 4
Party will be 240 minutes long


In [34]:
# Let's get this party rollingggg
env = simpy.Environment() # simulation environment
env.process(run_party(env))
env.run(until=time_minutes) # run until we reach our time limit


You play some Rocket League with Vishal and reach Grand Champion. Quit. Out. Of. Respect.
    Current time is 16
    You've had 0 drinks
    Your current bac is 0 ;)
You play Twister with Veijay and Vishal. You're all limbered up and ready to party
    Current time is 31
    You've had 0 drinks
    Your current bac is 0 ;)
The whole party plays Boom. Trevor gets the boom cup
    Current time is 61
    You've had 1 drinks
    Your current bac is 0.009943631050070545 ;)
We play Settlers of Catan. Your monopoly on bricks brings home the dub
    Current time is 121
    You've had 1 drinks
    Your current bac is -0.005056368949929457 ;)
The whole party plays Boom. You get the boom cup ¯\_(ツ)_/¯
    Current time is 151
    You've had 4 drinks
    Your current bac is 0.06302452420028218 ;)
Logan starts telling you about his favorite craft beer. You drink one to appease him
    Current time is 161
    You've had 4 drinks
    Your current bac is 0.060524524200282176 ;)
Pokemon Sorry! Innocent 